# Plan

Задача: бинарная сегментация

На вход двумерное одноканальное (grayscale) изображение, на выходе бинарная маска.

Про отключение dropout/batch normalization для инференса https://discuss.pytorch.org/t/model-eval-vs-with-torch-no-grad/19615

In [1]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
from pathlib import Path
import pandas as pd
import torch

In [2]:
class BraTSDataset(Dataset):
    def __init__(self, meta: pd.DataFrame, source_folder: [str, Path], transform=None):
        if isinstance(source_folder, str):
            source_folder = Path(source_folder)
            
        self.source_folder = source_folder
        self.meta_images = meta.query('is_mask == False').sort_values(by='sample_id').reset_index(drop=True)
        self.meta_masks = meta.query('is_mask == True').sort_values(by='sample_id').reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return self.meta_images.shape[0]

    def __getitem__(self, i):
        image = np.load(self.source_folder / self.meta_images.iloc[i]['relative_path'], allow_pickle=True)
        mask = np.load(self.source_folder / self.meta_masks.iloc[i]['relative_path'], allow_pickle=True)
        sample = image, mask
        
        if self.transform:
            image, mask = self.transform(sample)

        return torch.from_numpy(image).resize(1, 240, 240), torch.from_numpy(mask)

In [3]:
data_folder = Path('/home/anvar/work/data/brats_slices/')
df = pd.read_csv(data_folder / 'meta.csv', index_col=0)
data = BraTSDataset(df, data_folder)

In [4]:
len(data)

50844

In [5]:
data[10][0].shape

/home/anvar/anaconda3/lib/python3.8/site-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


torch.Size([1, 240, 240])

# 2. Define Unet architecture

https://arxiv.org/abs/1505.04597

![title](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png)


с одним изменением, вместо кропов и transpose convolutions будем интерполировать и использовать свертку (чтобы в результате на выходе получить маску того же размера что и вход).

In [6]:
import torch.nn as nn

In [7]:
def conv_3x3(in_c, out_c):
    return nn.Sequential(
                nn.Conv2d(in_c, out_c, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_c, out_c, kernel_size=3, padding=1),
                nn.ReLU(inplace=True)
    )

def up_conv(in_c, out_c):
    return nn.Sequential(      
        nn.Upsample(scale_factor=2, mode='bilinear'),
        nn.Conv2d(in_c, out_c, kernel_size=3, padding=1)
    )

class Unet(nn.Module):
    
    def __init__(self, ):
        super().__init__()
        
        self.max_pool2x2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.down_conv_1 = conv_3x3(1, 8)
        self.down_conv_2 = conv_3x3(8, 16)
        self.down_conv_3 = conv_3x3(16, 32)
        self.bottleneck_conv = conv_3x3(32, 64)
        
        self.upsample_1 = up_conv(64, 32)
        self.up_conv_1 = conv_3x3(64, 32)
        self.upsample_2 = up_conv(32, 16)
        self.up_conv_2 = conv_3x3(32, 16)
        self.upsample_3 = up_conv(16, 8)
        self.up_conv_3 = conv_3x3(16, 8)
        
        self.segm = nn.Sequential(
            nn.Conv2d(8, 8, kernel_size=3, padding=1),
            nn.Conv2d(8, 1, kernel_size=1)
        )
        
        
    def forward(self, x):
        
        # down/contracting
        x1 = self.down_conv_1(x) #
        x2 = self.max_pool2x2(x1)
        
        x3 = self.down_conv_2(x2)#
        x4 = self.max_pool2x2(x3)
        
        x5 = self.down_conv_3(x4)#
        x6 = self.max_pool2x2(x5)
        
        x7 = self.bottleneck_conv(x6)
    
        # up/expansive
        x = self.upsample_1(x7)
        x = self.up_conv_1(torch.cat([x, x5], axis=1))
        
        x = self.upsample_2(x)
        x = self.up_conv_2(torch.cat([x, x3], axis=1))
        
        x = self.upsample_3(x)
        x = self.up_conv_3(torch.cat([x, x1], axis=1))
        
        # segm
        x = self.segm(x)
        
        return x

In [8]:
net = Unet()

In [9]:
x = torch.rand(1,1,240,240)

In [10]:
net(x)

/home/anvar/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:3060: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn("Default upsampling behavior when mode={} is changed "


tensor([[[[-0.1394, -0.1332, -0.1312,  ..., -0.1341, -0.1327, -0.1289],
          [-0.1288, -0.1269, -0.1224,  ..., -0.1251, -0.1240, -0.1222],
          [-0.1311, -0.1246, -0.1249,  ..., -0.1231, -0.1224, -0.1198],
          ...,
          [-0.1291, -0.1279, -0.1236,  ..., -0.1242, -0.1227, -0.1212],
          [-0.1295, -0.1255, -0.1250,  ..., -0.1238, -0.1245, -0.1234],
          [-0.1310, -0.1272, -0.1269,  ..., -0.1264, -0.1269, -0.1323]]]],
       grad_fn=<MkldnnConvolutionBackward>)

In [11]:
data_folder = Path('/home/anvar/work/data/brats_slices/')
df = pd.read_csv(data_folder / 'meta.csv', index_col=0)
dataset = BraTSDataset(df, data_folder)

In [12]:
dataset_loader = torch.utils.data.DataLoader(dataset,
                                             batch_size=4, shuffle=True,
                                             num_workers=8)

In [13]:
import torch.optim as optim

In [14]:
device = ("cuda" if torch.cuda.is_available() else 'cpu')
model = Unet().to(device)

criterion = nn.BCEWithLogitsLoss() # 

optimizer = optim.Adam(model.parameters(), lr=0.01)

loader = dataset_loader

In [15]:
from tqdm.notebook import tqdm

In [16]:
for epoch in range(5):
    epoch_loss = 0
    for X_batch, y_batch in tqdm(loader):
        
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(X_batch.float())
        
        loss = criterion(y_pred, y_batch.float().unsqueeze(1))        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()        

    print(f'Epoch {epoch+0:03}: | Loss: {epoch_loss/len(dataset_loader):.5f}')

KeyboardInterrupt: 

# Вопросы на подумать

1. У такого способа задавать архитектуру Unet есть серьезный недостаток, несмотря на то что эта сеть полносверточная она не будет работать со входом произвольного размера, он обязательно должен делиться на 2 столько раз нацело, скольку у нас уровней юнета. Попробуйте например сделать (не меняя архитектуры):

```
x = torch.rand(1,1,220,220)
net(x)
```

Вопрос, как это можно исправить?

2. На каждом уровне up ветки мы конкатенируем картинку снизу, и картинку слева. Конкатенирвоать не обязательно, их можно просто складывать, такая сетка будет быстрее учиться, но на качество сегментации это влияет незначительно.

